In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from datetime import datetime
import os
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle
import importlib

from MODELS import LSTM_BEKK_MODEL, BEKK_GARCH_MODEL, DCC_GARCH_MODEL
from functions import *
from evaluate import *

In [2]:
def frobenius_loss(H_true: np.ndarray, H_pred: np.ndarray) -> float:
    """
    Frobenius norm loss: ||H_true - H_pred||_F^2
    """
    diff = H_true - H_pred
    return np.linalg.norm(diff, ord="fro")**2


def stein_loss(H_true: np.ndarray, H_pred: np.ndarray) -> float:
    """
    Stein loss (a likelihood-based measure):
    tr(H_true^{-1} H_pred) - log det(H_true^{-1} H_pred) - n
    """
    n = H_true.shape[0]
    try:
        inv_H = np.linalg.inv(H_true)
    except np.linalg.LinAlgError:
        # add jitter for numerical stability
        inv_H = np.linalg.inv(H_true + 1e-8 * np.eye(n))

    A = inv_H @ H_pred
    loss = np.trace(A) - np.log(np.linalg.det(A)) - n
    return np.real(loss)


def correlation_loss(H_true: np.ndarray, H_pred: np.ndarray, fisher_z: bool = False) -> float:
    """
    Frobenius loss between correlation matrices.
    Optionally applies Fisher-z transform for scale adjustment.
    """
    # convert to correlation matrices
    D_true = np.sqrt(np.diag(H_true))
    D_pred = np.sqrt(np.diag(H_pred))
    
    R_true = H_true / np.outer(D_true, D_true)
    R_pred = H_pred / np.outer(D_pred, D_pred)

    if fisher_z:
        # Fisher z-transform: z = 0.5 * ln((1+r)/(1-r))
        R_true = np.arctanh(np.clip(R_true, -0.999999, 0.999999))
        R_pred = np.arctanh(np.clip(R_pred, -0.999999, 0.999999))

    diff = R_true - R_pred
    return np.linalg.norm(diff, ord="fro")**2

def portfolio_aligned_loss(H_pred, H_true, weights):
    return ((weights.T @ H_pred @ weights) - (weights.T @ H_true @ weights))**2



Data cleaning:

Some close price is inappropriate, which lead to over the price fluctuation limit set by the exchange, fixed this by removing the observations that have day-to-day change over the exchange limit.

In [3]:
# ticker_list = ['REE', 'SAM', 'HAP', 'GMD', 'GIL', 'TMS', 'SAV', 'DHA', 'MHC', 'HAS'] # 10 stocks with the most observations
ticker_list = ['REE', 'SAM', 'HAP'] # 3 stocks with the most observations
limits = {
    'hose':0.07,
    'hnx':0.1,
    'upcom':0.15
}

# Holding period
horizon = 20


In [4]:
# Read and merge into 1 dataset

if "stock_data.csv" in os.listdir("data"):
    merged_df = pd.read_csv(
        os.path.join("data", "stock_data.csv"),
        index_col=None
    ).assign(
        date = lambda df : pd.to_datetime(df["date"])
    )
else:
    # Read and merge data
    hnx = pd.read_csv(os.path.join("data", "CafeF.HNX.Upto31.07.2025.csv")).assign(
        floor = "hnx"
    )
    hsx = pd.read_csv(os.path.join("data", "CafeF.HSX.Upto31.07.2025.csv")).assign(
        floor = "hose"
    )
    upcom = pd.read_csv(os.path.join("data", "CafeF.UPCOM.Upto31.07.2025.csv")).assign(
        floor = "upcom"
    )
    indexes = pd.read_csv(os.path.join("data", "CafeF.INDEX.Upto06.08.2025.csv")).assign(
        floor = "index"
    )

    # Rename columns
    hnx, hsx, upcom, indexes = [
        df.rename(columns={
            "<Ticker>":"ticker",
            "<DTYYYYMMDD>":"date",
            "<Open>":"open",
            "<High>":"high",
            "<Low>":"low",
            "<Close>":"close",
            "<Volume>":"volume"
        }) for df in [hnx, hsx, upcom, indexes]
    ]
        
    # Merge and clean data
    # UPCOM has missing tickers for some reason
    merged_df = pd.concat(
        [hnx, hsx, upcom, indexes],
        axis=0
    ).reset_index(drop=True).dropna(subset="ticker")\
    .assign(
        date=lambda df : df["date"].astype(str).apply(lambda x: datetime.strptime(x, "%Y%m%d").date())
    )
    merged_df.to_csv(
        os.path.join("data", "stock_data.csv"),
        index=False
    ) # Save merged data to save time in future runs


# Data cleaning and merging

data = merged_df[["date", "ticker", "floor", "close"]].sort_values(["ticker", "date"]).assign(
    returns = lambda df : df.groupby("ticker")["close"].pct_change(),
    log_returns_pct = lambda df : np.log(df["close"] / df.groupby("ticker")["close"].shift(1))*100
)

data = data.loc[data["ticker"].str.len()==3] # Eliminate ETF, and indeces

data["limit"] = data["floor"].map(limits)
outliers = data.loc[data["returns"].abs() > data["limit"]]
clean_df = data.drop(outliers.index) # Remove outliers
print(f"% of observations removed: {round((len(outliers)/len(data))*100, 2)}%")

# NOTE: try out different samples of stocks
pivoted_df = clean_df.pivot_table(values="returns", index="date", columns="ticker") # Pivot data for better usability
pivoted_df = pivoted_df[ticker_list].dropna()

display(pivoted_df.describe())
train_df, test_df = split_train_test(pivoted_df)

# Demean returns
train_mean = train_df.mean()
dm_train_df = train_df - train_mean
dm_test_df = test_df - train_mean

% of observations removed: 1.05%


ticker,REE,SAM,HAP
count,5951.000000,5951.000000,5951.000000
mean,0.001091,0.000699,0.000770
std,0.021411,0.023733,0.024894
min,-0.069971,-0.069999,-0.069963
25%,-0.009689,-0.011620,-0.012434
50%,0.000000,0.000000,0.000000
75%,0.011761,0.012037,0.012855
max,0.069962,0.069919,0.069927


Why demean the returns?

Volatility models like BEKK aim to model the covariance structure, not the mean.
By removing the mean from the return, it tells the model to focus on modeling volatility clustering and correlations, as well as preventing the mean return from contaminating the volatility dynamics

The mean from the training set will be used to demean the test set to simulate real world situation.

What's the differences between using static mean and moving average?

### ARCH
An ARCH model is used to predict volatility at a future time step, with the parameter $q$ as the number of lag squared residual error to include in the model 
ARCH uses returns or residuals as volatility shocks.

In [5]:
from arch import arch_model

garch = arch_model(pivoted_df["REE"]*100, vol="ARCH")
garch.fit()

Iteration:      1,   Func. Count:      5,   Neg. LLF: 129357.59275774604
Iteration:      2,   Func. Count:     12,   Neg. LLF: 15382.891572734194
Iteration:      3,   Func. Count:     19,   Neg. LLF: 13170.123600844268
Iteration:      4,   Func. Count:     24,   Neg. LLF: 12577.94370714599
Iteration:      5,   Func. Count:     28,   Neg. LLF: 12577.939139618782
Iteration:      6,   Func. Count:     32,   Neg. LLF: 12577.938832634314
Iteration:      7,   Func. Count:     36,   Neg. LLF: 12577.938830220966
Iteration:      8,   Func. Count:     39,   Neg. LLF: 12577.938830221014
Optimization terminated successfully    (Exit mode 0)
            Current function value: 12577.938830220966
            Iterations: 8
            Function evaluations: 39
            Gradient evaluations: 8


                      Constant Mean - ARCH Model Results                      
Dep. Variable:                    REE   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:               -12577.9
Distribution:                  Normal   AIC:                           25161.9
Method:            Maximum Likelihood   BIC:                           25182.0
                                        No. Observations:                 5951
Date:                Fri, Aug 22 2025   Df Residuals:                     5950
Time:                        14:43:52   Df Model:                            1
                                Mean Model                                
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0930  2.486e-02      3.740  1.837e-04 [4.426e-0

### GARCH

In [6]:
from arch import arch_model

garch = arch_model(pivoted_df["REE"]*100, vol="GARCH")
garch.fit()

Iteration:      1,   Func. Count:      6,   Neg. LLF: 6841595578418.32
Iteration:      2,   Func. Count:     15,   Neg. LLF: 6411162190.4786215
Iteration:      3,   Func. Count:     23,   Neg. LLF: 14540.272691777205
Iteration:      4,   Func. Count:     30,   Neg. LLF: 12675.511071518553
Iteration:      5,   Func. Count:     38,   Neg. LLF: 12163.288926570278
Iteration:      6,   Func. Count:     44,   Neg. LLF: 12162.193236000765
Iteration:      7,   Func. Count:     49,   Neg. LLF: 12162.192674535536
Iteration:      8,   Func. Count:     54,   Neg. LLF: 12162.192667947853
Iteration:      9,   Func. Count:     59,   Neg. LLF: 12162.192667437183
Optimization terminated successfully    (Exit mode 0)
            Current function value: 12162.192667437183
            Iterations: 9
            Function evaluations: 59
            Gradient evaluations: 9


                     Constant Mean - GARCH Model Results                      
Dep. Variable:                    REE   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -12162.2
Distribution:                  Normal   AIC:                           24332.4
Method:            Maximum Likelihood   BIC:                           24359.2
                                        No. Observations:                 5951
Date:                Fri, Aug 22 2025   Df Residuals:                     5950
Time:                        14:43:52   Df Model:                            1
                                Mean Model                                
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0747  2.140e-02      3.493  4.770e-04 [3.281e-0

# BEKK-GARCH

$$H_t=C'C+A'\epsilon_{t-1}\epsilon_{t-1}'+B'H_{t-1}B$$

- $H_t$: n_assets x n_assets conditional covariance matrix
- $C$: Lower triangular matrix to ensure positive definteness
- $A$: Captures the effect of past shocks (ARCH)
- $B$: Captures the persistence (GARCH)
- $\epsilon_{t-1}$: Vector of past residuals (demeaned returns)

### Usage
- Used for small dimensions of 2-3 assets.
- Contagion studies, volatility spillovers.

### Advantages
- Guarantees positive definite covariance matrices
- Captures spillover effects across assets
- More flexible in modeling asymmetric dependencies

### Disadvantages
- Computationally heavy, parameter explosion: for $N$ assets, get ~$N^2$ parameters
- Harder to estimate high-dimensional data
- May overfit with limited data

Since the model suffers from the curse of dimensionality, so I can only sample 3 stocks, at around 1000 days windows. This should be tuned later

In [7]:
# Run model
if "bekk_results.pkl" in os.listdir("bekk_params"):
    with open(os.path.join("bekk_params", "bekk_results.pkl"), "rb") as f:
        bekk = pickle.load(f)
    with open(os.path.join("bekk_params", "bekk_A.pkl"), "rb") as f:
        A = pickle.load(f)
    with open(os.path.join("bekk_params", "bekk_B.pkl"), "rb") as f:
        B = pickle.load(f)
    with open(os.path.join("bekk_params", "bekk_C.pkl"), "rb") as f:
        C = pickle.load(f) 
else:
    C, A, B, bekk = BEKK_GARCH_MODEL.fit_bekk(dm_train_df.tail(1000).values) # Fit BEKK-GARCH model, dimensions is greately reduced as this model suffers from curse of dimensionality
    # Save covariance matrix and model parameters
    with open(os.path.join("bekk_params", "bekk_results.pkl"), "wb") as f:
        pickle.dump(bekk, f)
    with open(os.path.join("bekk_params", "bekk_C.pkl"), "wb") as f:
        pickle.dump(C, f)
    with open(os.path.join("bekk_params", "bekk_A.pkl"), "wb") as f:
        pickle.dump(A, f)
    with open(os.path.join("bekk_params", "bekk_B.pkl"), "wb") as f:
        pickle.dump(B, f)

# Get BEKK-GARCH fitted covariance        
cov_matrix = BEKK_GARCH_MODEL.bekk_fitted_covariances(bekk.x, returns=dm_train_df.values)

In [8]:
# Evaluate
last_return = dm_train_df.iloc[-1].to_numpy()
last_cov_matrix = cov_matrix[-1]

dates_test = dm_test_df.index

bekk_port_returns, bekk_port_vars, bekk_act_covs = [], [], [] # Containers

# Loop over test period in 20-days non-overlapping horizons
for start in range(0, len(dm_test_df) - horizon + 1, horizon):
    train_data = pd.concat([dm_train_df, dm_test_df.iloc[:start]])
    
    # Forecast x step ahead
    cov_list = BEKK_GARCH_MODEL.bekk_forecast(
        C, A, B, 
        train_data.values, 
        horizon=horizon
    )

    # Aggregate to 20-days covariance forecast
    agg_covariance = sum(cov_list)

    # Get MVP weights
    mvp_weights, weights_dict = minimum_variance_portfolio(agg_covariance , train_df)

    # Realized returns from next 20-days
    horizon_return = test_df[start:start+horizon]


    # Cummulative return
    port_return = np.array(horizon_return) @ mvp_weights
    bekk_port_returns.append(port_return.sum())

    # Actual covariance
    act_covariance =  horizon_return.T @ horizon_return
    act_var = mvp_weights.T @ act_covariance @ mvp_weights
    bekk_port_vars.append(act_var)
    bekk_act_covs.append(act_covariance)

    # Adjust for next iteration
    last_return = np.array(horizon_return.iloc[[-1]])[0]
    last_cov_matrix = np.array(bekk_act_covs[-1])

bekk_garch_results =  pd.DataFrame({
    "date":dates_test[horizon-1::horizon][:len(bekk_port_returns)], # End of each horizon
    "realized_return":bekk_port_returns,
    "realized_variance":bekk_port_vars 
})

bekk_sr = bekk_garch_results["realized_return"].mean()/bekk_garch_results["realized_return"].std()
bekk_frob = frobenius_loss(H_pred=agg_covariance, H_true=act_covariance)
bekk_stein = stein_loss(H_pred=agg_covariance, H_true=act_covariance)
bekk_corr_loss = correlation_loss(H_pred=agg_covariance, H_true=act_covariance)
bekk_port_aligned = portfolio_aligned_loss(agg_covariance, act_covariance, mvp_weights)

print(f"""
BEKK-GARCH MODEL
      
- Sharpe Ratio = {bekk_sr}
- Frobenius norm = {bekk_frob}
- Stein loss = {bekk_stein}
- Correlation loss = {bekk_corr_loss}
- Portfolio aligned loss = {bekk_port_aligned}
""")

# Save results
with open(os.path.join("bekk_params", "bekk_portfolio_return.pkl"), "wb") as f:
    pickle.dump(bekk_port_returns, f)
with open(os.path.join("bekk_params", "bekk_portfolio_variance.pkl"), "wb") as f:
    pickle.dump(bekk_port_vars, f)
with open(os.path.join("bekk_params", "bekk_actual_covariance.pkl"), "wb") as f:
    pickle.dump(bekk_act_covs, f)
with open(os.path.join("bekk_params", "bekk_forecast_covariance.pkl"), "wb") as f:
    pickle.dump(agg_covariance, f)
with open(os.path.join("bekk_params", "bekk_weights.pkl"), "wb") as f:
    pickle.dump(weights_dict, f)


BEKK-GARCH MODEL
      
- Sharpe Ratio = 0.10774127440649622
- Frobenius norm = 5.166031022084115e-05
- Stein loss = 13.938248837048857
- Correlation loss = 6.2819259389772135
- Portfolio aligned loss = 2.7505339551703894e-06



# DCC-GARCH
Is a two-step model:
1. Estimate univariate GARCH for each asset's variance

$$h_{i,t}=\omega_i + \alpha_i \epsilon_{i,t-1}^2+\beta_i h_{i,t-1}$$

2. Model the correlations dynamically:

$$H_t=D_tR_tD_t$$

With:
- $D_t=diag(\sqrt{h_{1,t}},...,\sqrt{h_{N,t}})$: matrix of standard deviations
- $R_t$: dynamic correlation matrix, updated via:
$$Q_t=(1-\alpha-\beta)\overline{Q}+\alpha \epsilon_{t-1} \epsilon_{t-1}' + \beta Q_{t-1}$$
$$R_t=diag(Q_t)^{-\frac{1}{2}} Q_t diag(Q_t)^{-\frac{1}{2}}$$
- $\overline{Q}$: Unconditional correlation matrix

### Usage
- Designed for high-dimensional portfolios (10-100 assets)
- Used for correlation dynamics, portfolio optimization, hedging

### Advantages
- Computationally efficient, espically in large dimensions
- Decouples volatility (diagonal part) and correlation (off-diagonal part)
- Easier to interpret dynamic correlation structure
- Widely used in epirical finance

### Disadvantages
- Less flexible than BEKK
- Correlations may be biased if univariage GARCH models are misspecified
- Does not directly capture cross-variance spillovers


In [9]:
dm_train_df = dm_train_df.astype(float)
h_mat, eps_mat, garch_params = DCC_GARCH_MODEL.fit_univariate_garch(dm_train_df) # Fit univariate GARCH for each stock
dcc = DCC_GARCH_MODEL.fit_dcc(eps_mat) # Fit DCC(1,1) on standardized residuals
cov_matrix = DCC_GARCH_MODEL.build_covmatrix(h_mat, dcc["Rt"]) # Get full list of conditinoal covariance

In [10]:
# Evaluate

horizon = 20

last_return = dm_train_df.iloc[-1].to_numpy()
last_cov_matrix = cov_matrix[-1]

dates_test = dm_test_df.index

dcc_port_returns, dcc_port_vars, dcc_act_covs = [], [], [] # Containers

# Loop over test period in 20-days non-overlapping horizons
for start in range(0, len(dm_test_df) - horizon + 1, horizon):
    train_data = pd.concat([dm_train_df, dm_test_df.iloc[:start]])
    
    # Forecast x step ahead
    cov_list, _, _, _ = DCC_GARCH_MODEL.forecast_dcc_multi_step(
        h_last=h_mat[-1],
        r_last=last_return,
        garch_params=garch_params,
        eps_last=eps_mat[-1],
        Q_last=dcc["Qt"][-1],
        dcc_params=dcc,
        S=dcc["S"]
    )

    # Aggregate to 20-days covariance forecast
    agg_covariance = sum(cov_list)

    # Get MVP weights
    mvp_weights, weights_dict = minimum_variance_portfolio(agg_covariance , train_df)

    # Realized returns from next 20-days
    horizon_return = test_df[start:start+horizon]


    # Cummulative return
    port_return = np.array(horizon_return) @ mvp_weights
    dcc_port_returns.append(port_return.sum())

    # Actual covariance
    act_covariance =  horizon_return.T @ horizon_return
    act_var = mvp_weights.T @ act_covariance @ mvp_weights
    dcc_port_vars.append(act_var)
    dcc_act_covs.append(act_covariance)

    # Adjust for next iteration
    last_return = np.array(horizon_return.iloc[[-1]])[0]
    last_cov_matrix = np.array(dcc_act_covs[-1])

dcc_garch_results =  pd.DataFrame({
    "date":dates_test[horizon-1::horizon][:len(dcc_port_returns)], # End of each horizon
    "realized_return":dcc_port_returns,
    "realized_variance":dcc_port_vars 
})

dcc_sr = dcc_garch_results["realized_return"].mean()/dcc_garch_results["realized_return"].std()
dcc_frob = frobenius_loss(H_pred=agg_covariance, H_true=act_covariance)
dcc_stein = stein_loss(H_pred=agg_covariance, H_true=act_covariance)
dcc_corr_loss = correlation_loss(H_pred=agg_covariance, H_true=act_covariance)
dcc_port_aligned = portfolio_aligned_loss(agg_covariance, act_covariance, mvp_weights)

print(f"""
DCC-GARCH MODEL      

- Sharpe Ratio = {dcc_sr}
- Frobenius norm = {dcc_frob}
- Stein loss = {dcc_stein}
- Correlation loss = {dcc_corr_loss}
- Portfolio aligned loss = {dcc_port_aligned}
""")

# Save results
with open(os.path.join("dcc_results", "dcc_portfolio_return.pkl"), "wb") as f:
    pickle.dump(dcc_port_returns, f)
with open(os.path.join("dcc_results", "dcc_portfolio_variance.pkl"), "wb") as f:
    pickle.dump(dcc_port_vars, f)
with open(os.path.join("dcc_results", "dcc_actual_covariance.pkl"), "wb") as f:
    pickle.dump(dcc_act_covs, f)
with open(os.path.join("dcc_results", "dcc_forecast_covariance.pkl"), "wb") as f:
    pickle.dump(agg_covariance, f)
with open(os.path.join("dcc_results", "dcc_weights.pkl"), "wb") as f:
    pickle.dump(weights_dict, f)


DCC-GARCH MODEL      

- Sharpe Ratio = 0.14936500680575968
- Frobenius norm = 8.740778400773257e-05
- Stein loss = 10.998319241644367
- Correlation loss = 0.13916569628533665
- Portfolio aligned loss = 2.67880626484674e-06



# LSTM-BEKK

$$H_t = C'C + C_t'C_t + a r_{t-1} r_{t-1}' + b H_{t-1}$$

$C_t$ is dynamically updated through an LSTM network $\overline{C_t}=LSTM(h_{t-1}, r_{t-1})$, with $C_t=LowerTriangular(\overline{C_t})$

In [12]:
# LSTM training parameters
# Fit model

save_path = os.path.join("lstm_model", "lstm_bekk_model_state_dict.pt")

if "lstm_bekk_model_state_dict.pt" in os.listdir("lstm_model"):
    # Load model weights from file
    lstm_bekk_model = LSTM_BEKK_MODEL.load_model(
        path=save_path,
        n_assets=dm_train_df.shape[1],
        config=LSTM_BEKK_MODEL.LSTM_BEKK_config(
            hidden_size=3, # Same as number of assets
            num_layers=1,
            dropout=0.1,
            lr=0.001,
            epochs=600
        )
    )
    lstm_bekk_model.load_state_dict(
        torch.load(save_path)
    )
    lstm_bekk_model.eval()
else:
    
    lstm_bekk_model = LSTM_BEKK_MODEL.fit_lstm_bekk(
        returns_df=dm_train_df,
        hidden_size=3, # Same as number of assets
        num_layers=1,
        dropout=0.1, 
        lr=0.001,
        epochs=600
    )

    # Save model weights using torch.save
    torch.save(lstm_bekk_model.state_dict(), save_path)

[0000] train NLL : -552.873 | val NLL : -90.522
[0010] train NLL : -1667.422 | val NLL : -192.839
[0020] train NLL : -2128.580 | val NLL : -241.875
[0030] train NLL : -2459.073 | val NLL : -277.731
[0040] train NLL : -2731.052 | val NLL : -307.488
[0050] train NLL : -2969.590 | val NLL : -333.698
[0060] train NLL : -3186.289 | val NLL : -357.570
[0070] train NLL : -3387.506 | val NLL : -379.773
[0080] train NLL : -3577.058 | val NLL : -400.711
[0090] train NLL : -3757.364 | val NLL : -420.643
[0100] train NLL : -3930.009 | val NLL : -439.737
[0110] train NLL : -4096.049 | val NLL : -458.108
[0120] train NLL : -4256.342 | val NLL : -475.847
[0130] train NLL : -4411.469 | val NLL : -493.019
[0140] train NLL : -4561.981 | val NLL : -509.684
[0150] train NLL : -4708.358 | val NLL : -525.894
[0160] train NLL : -4851.046 | val NLL : -541.700
[0170] train NLL : -4990.369 | val NLL : -557.129
[0180] train NLL : -5126.995 | val NLL : -572.278
[0190] train NLL : -5261.032 | val NLL : -587.136
[0

In [13]:
# Evaluate model
cov_matrix = lstm_bekk_model.covariance(dm_train_df)
last_return = dm_train_df.iloc[-1].to_numpy()
last_cov_matrix = cov_matrix[-1]

### 20 stepts forecast
cov_matrix_20_steps = lstm_bekk_model.forecast_multi_step(
    last_returns=last_return,
    last_cov=last_cov_matrix,
    steps=20,
    method="zero"
)
# Evaluate MVP

last_return = dm_train_df.iloc[-1].to_numpy()
last_cov_matrix = cov_matrix[-1]

train_mean = train_df.mean()
demean_test_df = test_df - train_mean # Demean with in-sample mean
dates_test = demean_test_df.index

lstm_port_returns, lstm_port_vars, lstm_act_covs = [], [], [] # Containers

# Loop over test period in 20-days non-overlapping horizons
for start in range(0, len(demean_test_df) - horizon + 1, horizon):
    train_data = pd.concat([train_df, demean_test_df.iloc[:start]])
    
    # Forecast x step ahead
    cov_list = lstm_bekk_model.forecast_multi_step(
        last_returns=last_return,
        last_cov=last_cov_matrix,
        steps=horizon,
        method="zero"
    )

    # Aggregate to 20-days covariance forecast
    agg_covariance = sum(cov_list)

    # Get MVP weights
    mvp_weights, weights_dict = minimum_variance_portfolio(agg_covariance , train_df)

    # Realized returns from next 20-days
    horizon_return = test_df[start:start+horizon]

    # Cummulative return
    port_return = np.array(horizon_return) @ mvp_weights
    lstm_port_returns.append(port_return.sum())

    # Actual covariance
    act_covariance =  horizon_return.T @ horizon_return
    act_var = mvp_weights.T @ act_covariance @ mvp_weights
    lstm_port_vars.append(act_var)
    lstm_act_covs.append(act_covariance)

    # Adjust for next iteration
    last_return = np.array(horizon_return.iloc[[-1]])[0]
    last_cov_matrix = np.array(lstm_act_covs[-1])

lstm_bekk_results =  pd.DataFrame({
    "date":dates_test[horizon-1::horizon][:len(lstm_port_returns)], # End of each horizon
    "realized_return":lstm_port_returns,
    "realized_variance":lstm_port_vars 
})

lstm_sr = lstm_bekk_results["realized_return"].mean()/lstm_bekk_results["realized_return"].std()
lstm_frob = frobenius_loss(H_pred=agg_covariance, H_true=act_covariance)
lstm_stein = stein_loss(H_pred=agg_covariance, H_true=act_covariance)
lstm_corr_loss = correlation_loss(H_pred=agg_covariance, H_true=act_covariance)
lstm_port_aligned = portfolio_aligned_loss(agg_covariance, act_covariance, mvp_weights)

print(f"""
LSTM-BEKK MODEL

- Sharpe Ratio = {lstm_sr}
- Frobenius loss = {lstm_frob} 
- Correlation loss = {lstm_corr_loss}
- Portfolio aligned loss = {lstm_port_aligned}
""")

# Save results
with open(os.path.join("lstm_model", "lstm_portfolio_return.pkl"), "wb") as f:
    pickle.dump(lstm_port_returns, f)
with open(os.path.join("lstm_model", "lstm_portfolio_variance.pkl"), "wb") as f:
    pickle.dump(lstm_port_vars, f)
with open(os.path.join("lstm_model", "lstm_actual_covariance.pkl"), "wb") as f:
    pickle.dump(lstm_act_covs, f)
with open(os.path.join("lstm_model", "lstm_forecast_covariance.pkl"), "wb") as f:
    pickle.dump(agg_covariance, f)
with open(os.path.join("lstm_model", "lstm_weights.pkl"), "wb") as f:
    pickle.dump(weights_dict, f)


LSTM-BEKK MODEL

- Sharpe Ratio = 0.21235378272239538
- Frobenius loss = 145.88264649485924 
- Correlation loss = 0.9002514177151276
- Portfolio aligned loss = 11.845417304300398



In [14]:
eval = pd.DataFrame({
    "Ratios":["Sharpe ratio", "Frobenius loss", "Correlation loss", "Portflio aligned loss", "Stein loss"],
    "BEKK-GARCH": [round(i,4) for i in [bekk_sr, bekk_frob, bekk_corr_loss, bekk_port_aligned, bekk_stein]],
    "DCC-GARCH":[round(i, 4) for i in [dcc_sr, dcc_frob, dcc_corr_loss, dcc_port_aligned, dcc_stein]],
    "LSTM-BEKK":[round(i, 4) for i in [lstm_sr, lstm_frob, lstm_corr_loss, lstm_port_aligned, lstm_stein]],
})

In [15]:
eval

,Ratios,BEKK-GARCH,DCC-GARCH,LSTM-BEKK
0,Sharpe ratio,0.1077,0.1494,0.2124
1,Frobenius loss,0.0001,0.0001,145.8826
2,Correlation loss,6.2819,0.1392,0.9003
3,Portflio aligned loss,0.0000,0.0000,11.8454
4,Stein loss,13.9382,10.9983,4708.4329


In [16]:
eval.to_clipboard()